In [19]:
# Cell 1: environment and imports
# If you need to install packages, uncomment and run the pip lines once.
# !pip install pandas numpy scipy statsmodels matplotlib seaborn
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

alpha = 0.05   

In [20]:
# Cell 2: load the dataset (adjust path if needed)
data_path = "../data/MachineLearningRating_v3.txt"
df = pd.read_csv(data_path, sep="|", low_memory=False)

print("Loaded:", data_path)
print("Rows, cols:", df.shape)
df.head()

Loaded: ../data/MachineLearningRating_v3.txt
Rows, cols: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [21]:
by_zip = df.groupby('PostalCode').size().sort_values(ascending=False)
print("Smallest zip groups:", by_zip.tail(10))

Smallest zip groups: PostalCode
7340    1
3655    1
322     1
7760    1
7560    1
7350    1
2210    1
7463    1
7789    1
8584    1
dtype: int64


In [22]:
# Cell 3: KPI derivation
# Ensure numeric columns
for c in ['TotalPremium','TotalClaims']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

df['claim_flag'] = (df['TotalClaims'] > 0).astype(int)               
df['claim_severity'] = df['TotalClaims'].where(df['claim_flag']==1)
df['margin'] = df['TotalPremium'] - df['TotalClaims']

print("KPIs created. Summary:")
print("Total policies:", len(df))
print("Policies with claim:", df['claim_flag'].sum())
print("Columns include:", [c for c in ['claim_flag','claim_severity','margin'] if c in df.columns])


KPIs created. Summary:
Total policies: 1000098
Policies with claim: 2788
Columns include: ['claim_flag', 'claim_severity', 'margin']


In [23]:
# Cell 4: clean Province / PostalCode / Gender columns and show counts
# Normalize string columns safely
for col in ['Province','PostalCode','Gender','PostalCode']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace({'nan': np.nan})

print("Top Provinces:")
if 'Province' in df.columns:
    display(df['Province'].value_counts().head(20))
else:
    print("No Province column found.")

print("\nTop PostalCodes:")
if 'PostalCode' in df.columns:
    display(df['PostalCode'].value_counts().head(20))
else:
    print("No PostalCode column found.")

print("\nGender values:")
if 'Gender' in df.columns:
    display(df['Gender'].value_counts())
else:
    print("No Gender column found.")

Top Provinces:


Province
Gauteng          393865
Western Cape     170796
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52718
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64


Top PostalCodes:


PostalCode
2000    133498
122      49171
7784     28585
299      25546
7405     18518
458      13775
8000     11794
2196     11048
470      10226
7100     10161
1724     10107
4360      9730
302       9531
152       9423
7750      9408
1863      8655
1022      8476
4068      8234
400       6692
4001      6647
Name: count, dtype: int64


Gender values:


Gender
Not specified    940990
Male              42817
Female             6755
Name: count, dtype: int64

In [24]:
# Cell 5: helper functions
def print_decision(p_value, alpha=alpha):
    decision = "REJECT null hypothesis" if p_value < alpha else "FAIL TO REJECT null hypothesis"
    print(f"p-value = {p_value:.6f} | alpha = {alpha} -> {decision}")
    return decision

def normality_test(series):
    s = series.dropna()
    if len(s) < 3:
        return False, np.nan
    if len(s) <= 5000:
        stat, p = stats.shapiro(s)
        return p >= alpha, p
    else:
        stat, p = stats.kstest((s - s.mean())/s.std(ddof=0), 'norm')
        return p >= alpha, p

def equal_variance_test(*groups):
    groups = [g.dropna() for g in groups]
    if any(len(g)<2 for g in groups):
        return False, np.nan
    stat, p = stats.levene(*groups)
    return p >= alpha, p

def safe_independent_ttest(a,b):
    eq_var, p_levene = equal_variance_test(a,b)
    tstat, p = stats.ttest_ind(a.dropna(), b.dropna(), equal_var=eq_var)
    return tstat, p, eq_var, p_levene

In [25]:
# Cell 6: Province-level hypothesis testing
print("=== H0: No risk differences across Provinces ===\n")
if 'Province' not in df.columns:
    print("Province column not present. Aborting province tests.")
else:
    # 1) Claim Frequency: chi-square test
    contingency = pd.crosstab(df['Province'], df['claim_flag'])
    print("Contingency table (Province x claim_flag) sample:")
    display(contingency.head())
    chi2, p_chi, dof, exp = stats.chi2_contingency(contingency)
    print("\nClaim Frequency across Provinces (Chi-squared):")
    print_decision(p_chi)

    # 2) Claim Severity across provinces - collect groups with >=5 severity observations
    sev_groups = []
    sev_labels = []
    for prov, g in df.groupby('Province'):
        s = g['claim_severity'].dropna()
        if len(s) >= 5:
            sev_groups.append(s)
            sev_labels.append(prov)
    print(f"\nProvince groups used for severity test (n>=5): {len(sev_groups)}")
    if len(sev_groups) >= 2:
        normals = [normality_test(g)[0] for g in sev_groups]
        if all(normals):
            fstat, p_anova = stats.f_oneway(*sev_groups)
            print("ANOVA (claim severity across provinces):")
            print_decision(p_anova)
            if p_anova < alpha:
                # Tukey post-hoc
                subset = df.dropna(subset=['claim_severity','Province'])[['claim_severity','Province']]
                print("\nTukey HSD (pairwise differences):")
                print(pairwise_tukeyhsd(subset['claim_severity'], subset['Province']))
        else:
            hstat, p_kw = stats.kruskal(*sev_groups)
            print("Kruskal-Wallis (claim severity):")
            print_decision(p_kw)
    else:
        print("Not enough province groups with claim severity data to test severity differences.")

    # 3) Margin across provinces - use groups with at least 10 policies
    margin_groups = []
    margin_labels = []
    for prov, g in df.groupby('Province'):
        s = g['margin'].dropna()
        if len(s) >= 10:
            margin_groups.append(s)
            margin_labels.append(prov)
    print(f"\nProvince groups used for margin test (n>=10): {len(margin_groups)}")
    if len(margin_groups) >= 2:
        normals_m = [normality_test(g)[0] for g in margin_groups]
        if all(normals_m):
            fstat_m, p_anova_m = stats.f_oneway(*margin_groups)
            print("ANOVA (margin across provinces):")
            print_decision(p_anova_m)
            if p_anova_m < alpha:
                subset_m = df.dropna(subset=['margin','Province'])[['margin','Province']]
                print("\nTukey HSD (margin):")
                print(pairwise_tukeyhsd(subset_m['margin'], subset_m['Province']))
        else:
            hstat_m, p_kw_m = stats.kruskal(*margin_groups)
            print("Kruskal-Wallis (margin):")
            print_decision(p_kw_m)
    else:
        print("Not enough province groups with margin data to test margin differences.")

=== H0: No risk differences across Provinces ===

Contingency table (Province x claim_flag) sample:


claim_flag,0,1
Province,,
Eastern Cape,30286,50
Free State,8088,11
Gauteng,392543,1322
KwaZulu-Natal,169298,483
Limpopo,24769,67



Claim Frequency across Provinces (Chi-squared):
p-value = 0.000000 | alpha = 0.05 -> REJECT null hypothesis

Province groups used for severity test (n>=5): 9
Kruskal-Wallis (claim severity):
p-value = 0.000000 | alpha = 0.05 -> REJECT null hypothesis

Province groups used for margin test (n>=10): 9
Kruskal-Wallis (margin):
p-value = 0.000000 | alpha = 0.05 -> REJECT null hypothesis


In [26]:
# Cell 8: Margin differences across many postal codes (ANOVA / Kruskal) - use postal codes with n>=min_count
print("=== H0: No significant margin differences across Postal Codes (multi-group) ===\n")
if 'PostalCode' not in df.columns:
    print("PostalCode column missing.")
else:
    min_count = 20
    postal_groups = []
    labels = []
    for pc, g in df.groupby('PostalCode'):
        if len(g) >= min_count:
            postal_groups.append(g['margin'].dropna())
            labels.append(pc)
    print(f"Postal codes with >= {min_count} records: {len(labels)}")
    if len(postal_groups) < 2:
        print("Not enough postal-code groups to run multi-group test.")
    else:
        normals = [normality_test(g)[0] for g in postal_groups]
        if all(normals):
            fstat, p_anova = stats.f_oneway(*postal_groups)
            print("ANOVA (margin across postal codes):")
            print_decision(p_anova)
            if p_anova < alpha:
                subset = df[df['PostalCode'].isin(labels)][['margin','PostalCode']].dropna()
                print("\nTukey HSD (pairwise postal code differences):")
                print(pairwise_tukeyhsd(subset['margin'], subset['PostalCode']))
        else:
            hstat, p_kw = stats.kruskal(*postal_groups)
            print("Kruskal-Wallis (margin across postal codes):")
            print_decision(p_kw)

=== H0: No significant margin differences across Postal Codes (multi-group) ===

Postal codes with >= 20 records: 843
Kruskal-Wallis (margin across postal codes):
p-value = 0.000000 | alpha = 0.05 -> REJECT null hypothesis


In [27]:
# Cell 9: Gender tests (claim frequency, severity, margin)
print("=== H0: No significant risk difference between Women and Men ===\n")
if 'Gender' not in df.columns:
    print("Gender column not present. Aborting gender tests.")
else:
    # Normalize gender into male/female/other
    gcol = df['Gender'].astype(str).str.strip().str.lower()
    female_vals = set(['f','female','woman','women'])
    male_vals = set(['m','male','man','men'])
    df['gender_group'] = gcol.apply(lambda x: 'female' if x in female_vals else ('male' if x in male_vals else 'other'))

    print("Gender counts:")
    display(df['gender_group'].value_counts())

    # Keep only male/female
    gender_df = df[df['gender_group'].isin(['female','male'])]
    female = gender_df[gender_df['gender_group']=='female']
    male = gender_df[gender_df['gender_group']=='male']

    # Claim frequency z-test
    cf_f, n_f = female['claim_flag'].sum(), len(female)
    cf_m, n_m = male['claim_flag'].sum(), len(male)
    print(f"\nClaim frequency: female {cf_f}/{n_f} vs male {cf_m}/{n_m}")
    if n_f==0 or n_m==0:
        print("Insufficient gender counts for z-test.")
    else:
        stat_g, p_g = proportions_ztest([cf_f, cf_m], [n_f, n_m])
        print("Proportions z-test (claim frequency by gender):")
        print_decision(p_g)

    # Severity comparison
    sev_f = female['claim_severity'].dropna()
    sev_m = male['claim_severity'].dropna()
    if len(sev_f) < 3 or len(sev_m) < 3:
        print("\nToo few severity samples in one of the gender groups for reliable test.")
    else:
        nf, nm = len(sev_f), len(sev_m)
        nf_norm, pnf = normality_test(sev_f)
        nm_norm, pnm = normality_test(sev_m)
        print(f"\nSeverity normality: female -> {nf_norm} (p={pnf}), male -> {nm_norm} (p={pnm})")
        if nf_norm and nm_norm:
            t_s, p_s, eqv_s, lv_s = safe_independent_ttest(sev_f, sev_m)
            print("T-test for severity by gender:")
            print(f"Levene p={lv_s:.6f}, equal_var={eqv_s}")
            print_decision(p_s)
        else:
            u_s, p_mw_s = stats.mannwhitneyu(sev_f, sev_m, alternative='two-sided')
            print("Mann-Whitney for severity by gender:")
            print_decision(p_mw_s)

    # Margin comparison
    m_f = female['margin'].dropna()
    m_m = male['margin'].dropna()
    if len(m_f) < 5 or len(m_m) < 5:
        print("\nToo few margin records in one of the gender groups for robust test.")
    else:
        nmf, pmf = normality_test(m_f)
        nmm, pmm = normality_test(m_m)
        if nmf and nmm:
            t_mg, p_mg, eqv_mg, lv_mg = safe_independent_ttest(m_f, m_m)
            print("\nT-test for margin by gender:")
            print(f"Levene p={lv_mg:.6f}, equal_var={eqv_mg}")
            print_decision(p_mg)
        else:
            u_mg, p_mw_mg = stats.mannwhitneyu(m_f, m_m, alternative='two-sided')
            print("\nMann-Whitney for margin by gender:")
            print_decision(p_mw_mg)

=== H0: No significant risk difference between Women and Men ===

Gender counts:


gender_group
other     950526
male       42817
female      6755
Name: count, dtype: int64


Claim frequency: female 14/6755 vs male 94/42817
Proportions z-test (claim frequency by gender):
p-value = 0.840494 | alpha = 0.05 -> FAIL TO REJECT null hypothesis

Severity normality: female -> True (p=0.11092318701431175), male -> False (p=1.2189369420515468e-15)
Mann-Whitney for severity by gender:
p-value = 0.223513 | alpha = 0.05 -> FAIL TO REJECT null hypothesis

Mann-Whitney for margin by gender:
p-value = 0.000000 | alpha = 0.05 -> REJECT null hypothesis


In [28]:
# Cell 10: If you ran many tests and want to control family-wise error, use Bonferroni
# Example: if you run k independent hypothesis tests, adjusted_alpha = alpha / k
k = 6   # change k to actual number of tests you've run
adjusted_alpha = alpha / k
print(f"Bonferroni adjusted alpha for {k} tests: {adjusted_alpha:.6f}")
print("If you want FDR (Benjamini-Hochberg), use statsmodels.sandbox or custom code.")

Bonferroni adjusted alpha for 6 tests: 0.008333
If you want FDR (Benjamini-Hochberg), use statsmodels.sandbox or custom code.


In [29]:
# Cell 11: generate a scaffold report (update manually with p-values & decisions from outputs)
report_lines = []
report_lines.append("# Task 3 - Hypothesis Testing Results\n")
report_lines.append("**Dataset:** data/MachineLearningRating_v3.txt\n")
report_lines.append("**Alpha:** 0.05\n\n")

report_lines.append("## Tests performed\n")
report_lines.append("- Claim Frequency by Province (Chi-squared)\n")
report_lines.append("- Claim Severity by Province (ANOVA / Kruskal-Wallis)\n")
report_lines.append("- Margin by Province (ANOVA / Kruskal-Wallis)\n")
report_lines.append("- Postal code pairwise comparisons (top 2 by count) — proportions z-test, severity & margin tests\n")
report_lines.append("- Margin across postal codes (ANOVA / Kruskal-Wallis)\n")
report_lines.append("- Gender comparisons: frequency (z-test), severity (t/Mann-Whitney), margin (t/Mann-Whitney)\n\n")

report_lines.append("## Summary - Fill with decisions from notebook outputs\n")
report_lines.append("- Provinces (frequency): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Provinces (severity): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Provinces (margin): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Postal codes (frequency top2): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Postal codes (margin top2): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Postal codes (margin multi-group): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Gender (frequency): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Gender (severity): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n")
report_lines.append("- Gender (margin): REPLACE_WITH_DECISION (p = REPLACE_WITH_P)\n\n")

report_lines.append("## Business Recommendations (example templates)\n")
report_lines.append("- If you reject H0 for Province frequency/severity: \"We REJECT H0 (p < 0.05). Province X shows higher claim frequency/severity — recommend revising premiums or underwriting criteria in Province X.\"\n")
report_lines.append("- If you reject H0 for PostalCode margin: \"We REJECT H0 (p < 0.05). Postal code Y shows lower margins — consider investigating local risk factors and adjusting pricing or marketing strategy.\"\n")
report_lines.append("- If you find gender differences: \"We REJECT H0 (p < 0.05). Differences between genders exist for KPI Z — evaluate potential causes and ensure non-discriminatory underwriting.\"\n")

# Save markdown
out_path = "../reports/Task3_HypothesisTesting_Report.md"
import os
os.makedirs(os.path.dirname(out_path), exist_ok=True)
with open(out_path, "w", encoding="utf-8") as f:
    f.writelines([ln+"\n" for ln in report_lines])

print("Scaffold report saved to", out_path)

Scaffold report saved to ../reports/Task3_HypothesisTesting_Report.md
